In [1]:
import torch
from IF.train_model_mnist import *
from proutils import *
import numpy as np
import kmedoids
from joblib import Parallel, delayed
from scipy.spatial.distance import cdist
from sklearn.metrics import accuracy_score, silhouette_score
from sklearn.metrics.pairwise import euclidean_distances
import matplotlib.pyplot as plt
from tqdm import tqdm
from aix360.algorithms.protodash import ProtodashExplainer
from scipy.spatial.distance import cosine
from torch.utils.data import TensorDataset, DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from collections import Counter

model = Net()
model.load_state_dict(torch.load('data/model_mnist.pth'))
model.eval()  # Set the model to evaluation mode
ifem=np.load("data/influence_scores_mnist.npy")
dmem=torch.load('data/embedsmnist_DM.pt')

transform = transforms.ToTensor()
train_dataset = datasets.MNIST(root='data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='data', train=False, download=True, transform=transform)

# Extract images and labels
train_images = torch.stack([train_dataset[i][0] for i in range(len(train_dataset))])  # [N_train, 1, 28, 28]
train_embeddings=model(train_images, emd=True).detach().numpy()
train_labels = torch.tensor([train_dataset[i][1] for i in range(len(train_dataset))])
test_images = torch.stack([test_dataset[i][0] for i in range(len(test_dataset))])    # [N_test, 1, 28, 28]
test_embeddings=model(test_images, emd=True).detach().numpy()
test_labels = torch.tensor([test_dataset[i][1] for i in range(len(test_dataset))])

/tmp/ipykernel_3081412/1235930754.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('data/model_mnist.pth'))
/tmp/ipykernel_3081412/12359

In [2]:
def surrogate_fidelity(prototypes, X_test, mod_pred):
    train_dataset = TensorDataset(X_test, mod_pred)
    trainloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    smodel = fit_model(trainloader)

    # Evaluate the model on test data
    smodel.eval()
    with torch.no_grad():
        outputs = torch.argmax(smodel(X_test), dim=1)
        accuracy = (outputs == mod_pred).sum().item() / len(mod_pred)
    return accuracy

In [3]:
aide_em=aide(ifem, train_embeddings, test_embeddings, 30, coverage=True)
prs=find_representative_samples(test_embeddings, train_embeddings, ifem, 20, 30, alpha=0.6)
distances = cosine_similarity(test_embeddings, test_embeddings[prs])
nearest_medoid_indices = np.argmax(distances, axis=1)

# def coverage(N, bin=False, popularity=True):
#     covs=[]
#     for i in np.unique(nearest_medoid_indices):
#         idx=np.where(nearest_medoid_indices == i)[0]
#         if popularity:
#             gx=[i[0] for i in Counter(np.array(aide_em)[idx].flatten()).most_common(60)]
#         else:
#             gx=aide_em[prs[i]]
#         for j in idx:
#             if bin:
#                 if len(set(aide_em[j]).intersection(set(gx)))>N:
#                     covs.append(1)
#                 else:
#                     covs.append(0)
#             else:
#                 covs.append(len(set(aide_em[j]).intersection(set(gx)))/len(aide_em[j]))
#     return sum(covs)/len(covs)

# covs= [coverage(i, bin=True, popularity=False) for i in range(31)]

connection matrix is ready


/home/cs.aau.dk/em63by/anaconda3/envs/mglop/lib/python3.8/site-packages/scipy/sparse/linalg/_eigen/arpack/arpack.py:1600: RuntimeWarning: k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.
  warnings.warn("k >= N for N * N square matrix. "


ValueError: n_samples=2 should be >= n_clusters=20.

In [12]:
dmem.shape

torch.Size([10000, 60000])

In [ ]:
#Sensitivity to the number of influential sample to build the graph with
mglop=Parallel(n_jobs=-1)(delayed(find_representative_samples)(test_embeddings, train_embeddings, ifem,20, i, alpha=0.6) for i in tqdm(range(5,41,5)))
accs=[]
fids=[]
sil=[]
sim=[]
def compute_metrics(i):
    acc = nearest_medoid_accuracy(test_embeddings, mod_pred, mglop[i])
    fid=surrogate_fidelity(mglop[i], test_images, mod_pred)
    sil = compute_prototype_silhouette_score(test_embeddings, mglop[i])
    labels = cluster_by_prototypes(test_embeddings, mglop[i])
    sim = expected_inter_cluster_similarity(test_embeddings, labels)
    return acc,fid, sil, sim

# Parallel computation
results = Parallel(n_jobs=64)(delayed(compute_metrics)(i) for i in range(len(mglop)))

# Unpack results into separate lists
accs,fids, sil, sim = zip(*results)
plt.plot(range(5,41,5), accs, marker='D', label='Faithfulness')
plt.plot(range(5,41,5), fids, marker='*', label='Fidelity')
plt.plot(range(5,41,5), sil, marker='x', label='Silhouette')
plt.plot(range(5,41,5), sim, marker='o', label='Expected similarity')
plt.legend()
plt.show()

In [ ]:
N_values=range(10,51, 5)
def generate_prototypes(N):
    explainer = ProtodashExplainer()
    weights, protodash, _ = explainer.explain(test_embeddings, test_embeddings, m=N, kernelType='euclid')
    
    return {
        "mglop": find_representative_samples(test_embeddings, train_embeddings, ifem, N, 15, alpha=0.6),
        "dknn": find_prototypes(test_embeddings, mod_pred, N),
        "dm": find_prototypes(dmem, mod_pred, N),
        "protodash": protodash
    }

n_jobs = -1 
all_protos = Parallel(n_jobs=n_jobs)(
    delayed(generate_prototypes)(N)
    for N in tqdm(N_values)
)

In [ ]:
# Intersection

algorithms = ['mglop', 'dknn', 'dm', 'protodash']
intersections = {f"{algo1} ∩ {algo2}": [] for i, algo1 in enumerate(algorithms) for algo2 in algorithms[i + 1:]}

# Extract intersections between algorithm outputs
for i, outputs in enumerate(all_protos):
    for j, algo1 in enumerate(algorithms):
        for algo2 in algorithms[j + 1:]:
            intersection_size = len(set(outputs[algo1]) & set(outputs[algo2]))
            intersections[f"{algo1} ∩ {algo2}"].append(intersection_size)

# Plotting the results
line_styles = ['-', '--', '-.', ':', '-', '--']
markers = ['o', 's', 'D', '^', 'v', 'P']

plt.figure(figsize=(12, 8))
for (label, sizes), linestyle, marker in zip(intersections.items(), line_styles, markers):
    plt.plot(N_values, sizes, linestyle=linestyle, marker=marker, label=label)

plt.xlabel("Number of Prototypes (N)")
plt.ylabel("Intersection Size")
plt.title("Intersection Size vs Number of Elements (N) for Algorithm Outputs")
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
plt.legend()
plt.grid(True)

# Save the plot as a PDF
# plt.savefig("Figures/intersection_plot.pdf", format="pdf")

# Show the plot
plt.show()

In [ ]:
all_protos_np=np.array(all_protos)

In [ ]:
from sklearn.metrics import adjusted_rand_score
from scipy.spatial.distance import cdist

def assign_clusters(X, prototypes):

    # Compute distance from each point to each prototype
    distances = cdist(X, prototypes, metric='euclidean')
    
    # Assign each point to the cluster with minimum distance
    labels = np.argmin(distances, axis=1)
    return labels

pairwise_aris = {
    "MGloP-DkNN": [],
    "MGloP-DM": [],
    "MGloP-PDash": [],
    "DkNN-DM": [],
    "DkNN-PDash": [],
    "DM-PDash": []
}

for i in range(len(all_protos_np)):
    labels1 = assign_clusters(test_embeddings, test_embeddings[list(all_protos_np[i].values())[0]])
    labels2 = assign_clusters(test_embeddings, test_embeddings[list(all_protos_np[i].values())[1]])
    labels3 = assign_clusters(test_embeddings, test_embeddings[list(all_protos_np[i].values())[2]])
    labels4 = assign_clusters(test_embeddings, test_embeddings[list(all_protos_np[i].values())[3]])
    
    
    # Compute ARIs for all pairs
    ari_12 = adjusted_rand_score(labels1, labels2)
    ari_13 = adjusted_rand_score(labels1, labels3)
    ari_14 = adjusted_rand_score(labels1, labels4)
    ari_23 = adjusted_rand_score(labels2, labels3)
    ari_24 = adjusted_rand_score(labels2, labels4)
    ari_34 = adjusted_rand_score(labels3, labels4)
    
    # Append results
    pairwise_aris["MGloP-DkNN"].append(ari_12)
    pairwise_aris["MGloP-DM"].append(ari_13)
    pairwise_aris["MGloP-PDash"].append(ari_14)
    pairwise_aris["DkNN-DM"].append(ari_23)
    pairwise_aris["DkNN-PDash"].append(ari_24)
    pairwise_aris["DM-PDash"].append(ari_34)

# Plot the ARI evolution for each pair
plt.figure(figsize=(10, 6))
for pair_name, ari_values in pairwise_aris.items():
    plt.plot(N_values, ari_values, marker='o', label=pair_name)

plt.title('ARI Evolution for Each Pair as Number of Prototypes Increases')
plt.xlabel('Number of Prototypes (k)')
plt.ylabel('ARI')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
mglop_ev=[]
dmodels_ev=[]
protodash_ev=[]
dknn_ev=[]
for i in range(len(all_protos_np)):
    mglop_ev.append([nearest_medoid_accuracy(test_embeddings, mod_pred, list(all_protos_np[i].values())[0]), compute_prototype_silhouette_score(test_embeddings, list(all_protos_np[i].values())[0]), surrogate_fidelity(list(all_protos_np[i].values())[0], test_images, mod_pred)])
    dknn_ev.append([nearest_medoid_accuracy(test_embeddings, mod_pred, list(all_protos_np[i].values())[1]), compute_prototype_silhouette_score(test_embeddings, list(all_protos_np[i].values())[1]),surrogate_fidelity(list(all_protos_np[i].values())[1], test_images, mod_pred)])
    dmodels_ev.append([nearest_medoid_accuracy(test_embeddings, mod_pred, list(all_protos_np[i].values())[2]), compute_prototype_silhouette_score(test_embeddings, list(all_protos_np[i].values())[2]), surrogate_fidelity(list(all_protos_np[i].values())[2], test_images, mod_pred)])
    protodash_ev.append([nearest_medoid_accuracy(test_embeddings, mod_pred, list(all_protos_np[i].values())[3]), compute_prototype_silhouette_score(test_embeddings, list(all_protos_np[i].values())[3]), surrogate_fidelity(list(all_protos_np[i].values())[3], test_images, mod_pred)])

In [ ]:
N_range=range(10,51,5)
plt.figure(figsize=(12, 8))
plt.plot(N_range, [item[2] for item in mglop_ev], marker='o', linestyle='-', label='MGLop Accuracy')
plt.plot(N_range, [item[2] for item in dknn_ev], marker='s', linestyle='--', label='DKNN Accuracy')
plt.plot(N_range, [item[2] for item in dmodels_ev], marker='D', linestyle='-.', label='DModels Accuracy')
plt.plot(N_range, [item[2] for item in protodash_ev], marker='^', linestyle=':', label='Protodash Accuracy')

plt.xlabel("Number of Prototypes (N)")
plt.ylabel("Fidelity")
plt.title("Surrogate Model Accuracy vs Number of Prototypes (N) for Different Algorithms")
plt.legend()
plt.grid(True)
# plt.savefig("Figures/nearest_medoid_accuracy_plot.pdf", format="pdf")
plt.show()

In [ ]:
N_range=range(10,51,5)
plt.figure(figsize=(12, 8))
plt.plot(N_range, [item[0] for item in mglop_ev], marker='o', linestyle='-', label='MGLop Accuracy')
plt.plot(N_range, [item[0] for item in dknn_ev], marker='s', linestyle='--', label='DKNN Accuracy')
plt.plot(N_range, [item[0] for item in dmodels_ev], marker='D', linestyle='-.', label='DModels Accuracy')
plt.plot(N_range, [item[0] for item in protodash_ev], marker='^', linestyle=':', label='Protodash Accuracy')

plt.xlabel("Number of Prototypes (N)")
plt.ylabel("Nearest Medoid Accuracy")
plt.title("Nearest Medoid Accuracy vs Number of Prototypes (N) for Different Algorithms")
plt.legend()
plt.grid(True)
# plt.savefig("Figures/nearest_medoid_accuracy_plot.pdf", format="pdf")
plt.show()

#### Stability (Silhouette)

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(N_range, [item[1] for item in mglop_ev], marker='o', linestyle='-', label='MGLop Silhouette Score')
plt.plot(N_range, [item[1] for item in dknn_ev], marker='s', linestyle='--', label='DKNN Silhouette Score')
plt.plot(N_range, [item[1] for item in dmodels_ev], marker='D', linestyle='-.', label='DModels Silhouette Score')
plt.plot(N_range, [item[1] for item in protodash_ev], marker='^', linestyle=':', label='Protodash Silhouette Score')

plt.xlabel("Number of Prototypes (N)")
plt.ylabel("Silhouette Score")
plt.title("Silhouette Score vs Number of Prototypes (N) for Different Algorithms")
plt.legend()
plt.grid(True)
# plt.savefig("Figures/silhouette_score_plot.pdf", format="pdf")
plt.show()

#### Expected Inter-cluster similarity

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist


# Experiment with increasing number of prototypes
num_prototypes_list = range(10, 51, 5)
avg_similarities = []
avg_similarities1 = []
avg_similarities2 = []
avg_similarities3 = []

for i in tqdm(range(9)):
    mglop=list(all_protos_np[i].values())[0]
    dmodels=list(all_protos_np[i].values())[2]
    dknn=list(all_protos_np[i].values())[1]
    protodash=list(all_protos_np[i].values())[3]
    labels = cluster_by_prototypes(test_embeddings, mglop)
    labels1 = cluster_by_prototypes(test_embeddings, dknn)
    labels2 = cluster_by_prototypes(test_embeddings, dmodels)
    labels3 = cluster_by_prototypes(test_embeddings, protodash)
    avg_similarity = expected_inter_cluster_similarity(test_embeddings, labels)
    avg_similarity1 = expected_inter_cluster_similarity(test_embeddings, labels1)
    avg_similarity2 = expected_inter_cluster_similarity(test_embeddings, labels2)
    avg_similarity3 = expected_inter_cluster_similarity(test_embeddings, labels3)
    avg_similarities.append(avg_similarity)
    avg_similarities1.append(avg_similarity1)
    avg_similarities2.append(avg_similarity2)
    avg_similarities3.append(avg_similarity3)

# Plot the results
plt.figure(figsize=(10, 6))
plt.plot(num_prototypes_list, avg_similarities, marker='o', label='MGloP')
plt.plot(num_prototypes_list, avg_similarities1, marker='x', label='DkNN')
plt.plot(num_prototypes_list, avg_similarities2, marker='*', label='Dmodels')
plt.plot(num_prototypes_list, avg_similarities3, marker='D', label='ProtoDash')
plt.xlabel('Number of Prototypes')
plt.ylabel('Expected Inter-Cluster Similarity')
plt.title('Expected Inter-Cluster Similarity vs Number of Prototypes')
plt.legend()
plt.grid(True)
plt.show()
